# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [7]:
# Import the WeatherPy_database.csv file. 
input_data = 'WeatherPy_Database.csv'
city_data_df = pd.read_csv(input_data)

# Display sample data
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Current_Description
0,0,Mataura,NZ,-46.1927,168.8643,59.41,52,33,5.19,scattered clouds
1,1,Vaini,TO,-21.2000,-175.2000,75.36,94,100,9.22,moderate rain
2,2,Selikhino,RU,50.3672,137.4996,2.43,91,100,12.91,overcast clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,46.51,71,75,9.22,light rain
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,72.30,93,100,2.28,overcast clouds
5,5,Cascais,PT,38.6979,-9.4215,62.51,95,100,11.99,overcast clouds
6,6,Botevgrad,BG,42.9000,23.7833,40.53,83,12,5.12,few clouds
7,7,Valparaiso,CL,-33.0393,-71.6273,60.44,88,100,9.22,overcast clouds
8,8,Lebu,CL,-37.6167,-73.6500,56.46,92,22,20.29,few clouds
9,9,Erseke,AL,40.3378,20.6789,38.34,92,89,1.59,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [8]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = int(input ("What is the minimum temperature you would like for your vacation?:"))
max_temp = int(input ("What is the maximum temperature you would like for your vacation?:"))

What is the minimum temperature you would like for your vacation?:72
What is the maximum temperature you would like for your vacation?:90


In [9]:
print(min_temp)
type(min_temp)

72


int

## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [10]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
#preferred_cities_df = city_data_df.loc[lambda row:row['Max Temp']> min_temp]
#df.loc[(df.age >= 25) & (df.address == 'Hanoi')] 
preferred_cities_df = city_data_df.loc[(city_data_df.Max_Temp >= min_temp)& (city_data_df.Max_Temp <= max_temp)]
# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Current_Description
1,1,Vaini,TO,-21.2000,-175.2000,75.36,94,100,9.22,moderate rain
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,72.30,93,100,2.28,overcast clouds
11,11,Busselton,AU,-33.6500,115.3333,83.32,41,0,15.48,clear sky
17,17,Maridi,SS,4.9158,29.4742,81.61,28,11,7.49,few clouds
21,21,Hwange,ZW,-18.3693,26.5019,83.57,50,83,8.61,broken clouds
26,26,Butterworth,MY,5.3991,100.3638,89.64,40,20,11.50,few clouds
34,34,Rikitea,PF,-23.1203,-134.9692,77.56,73,75,4.79,broken clouds
42,42,Cockburn Town,TC,21.4612,-71.1419,78.37,76,0,22.26,clear sky
44,44,Kismayo,SO,-0.3582,42.5454,83.80,60,4,18.07,clear sky
46,46,Alindao,CF,5.0267,21.2088,79.11,31,92,4.85,overcast clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [11]:
preferred_cities_df.isna().sum()

City_ID                0
City                   0
Country                3
Lat                    0
Lng                    0
Max_Temp               0
Humidity               0
Cloudiness             0
Wind_Speed             0
Current_Description    0
dtype: int64

In [12]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities_df = preferred_cities_df.dropna()

# Display sample data

clean_travel_cities_df.head(20)
#clean_travel_cities_df.isna().sum()

,City_ID,City,Country,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Current_Description
1,1,Vaini,TO,-21.2000,-175.2000,75.36,94,100,9.22,moderate rain
4,4,Arraial Do Cabo,BR,-22.9661,-42.0278,72.30,93,100,2.28,overcast clouds
11,11,Busselton,AU,-33.6500,115.3333,83.32,41,0,15.48,clear sky
17,17,Maridi,SS,4.9158,29.4742,81.61,28,11,7.49,few clouds
21,21,Hwange,ZW,-18.3693,26.5019,83.57,50,83,8.61,broken clouds
26,26,Butterworth,MY,5.3991,100.3638,89.64,40,20,11.50,few clouds
34,34,Rikitea,PF,-23.1203,-134.9692,77.56,73,75,4.79,broken clouds
42,42,Cockburn Town,TC,21.4612,-71.1419,78.37,76,0,22.26,clear sky
44,44,Kismayo,SO,-0.3582,42.5454,83.80,60,4,18.07,clear sky
46,46,Alindao,CF,5.0267,21.2088,79.11,31,92,4.85,overcast clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [23]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.

hotel_df = clean_travel_cities_df [['City','Country','Max_Temp','Lat','Lng',"Current_Description"]].copy()
hotel_df.head(10)

,City,Country,Max_Temp,Lat,Lng,Current_Description
1,Vaini,TO,75.36,-21.2000,-175.2000,moderate rain
4,Arraial Do Cabo,BR,72.30,-22.9661,-42.0278,overcast clouds
11,Busselton,AU,83.32,-33.6500,115.3333,clear sky
17,Maridi,SS,81.61,4.9158,29.4742,few clouds
21,Hwange,ZW,83.57,-18.3693,26.5019,broken clouds
26,Butterworth,MY,89.64,5.3991,100.3638,few clouds
34,Rikitea,PF,77.56,-23.1203,-134.9692,broken clouds
42,Cockburn Town,TC,78.37,21.4612,-71.1419,clear sky
44,Kismayo,SO,83.80,-0.3582,42.5454,clear sky
46,Alindao,CF,79.11,5.0267,21.2088,overcast clouds


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [24]:
# Create a new column "Hotel Name".
hotel_df['Hotel Name']=''
# Display sample data
hotel_df.head(10)

,City,Country,Max_Temp,Lat,Lng,Current_Description,Hotel Name
1,Vaini,TO,75.36,-21.2000,-175.2000,moderate rain,
4,Arraial Do Cabo,BR,72.30,-22.9661,-42.0278,overcast clouds,
11,Busselton,AU,83.32,-33.6500,115.3333,clear sky,
17,Maridi,SS,81.61,4.9158,29.4742,few clouds,
21,Hwange,ZW,83.57,-18.3693,26.5019,broken clouds,
26,Butterworth,MY,89.64,5.3991,100.3638,few clouds,
34,Rikitea,PF,77.56,-23.1203,-134.9692,broken clouds,
42,Cockburn Town,TC,78.37,21.4612,-71.1419,clear sky,
44,Kismayo,SO,83.80,-0.3582,42.5454,clear sky,
46,Alindao,CF,79.11,5.0267,21.2088,overcast clouds,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [25]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [26]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Vaini - nearest hotel: Keleti International Resort
Arraial Do Cabo - nearest hotel: No hotel found
Busselton - nearest hotel: No hotel found
Maridi - nearest hotel: No hotel found
Hwange - nearest hotel: No hotel found
Butterworth - nearest hotel: No hotel found
Rikitea - nearest hotel: Chez Bianca & Benoit
Cockburn Town - nearest hotel: The Salt Raker Inn
Kismayo - nearest hotel: Kismayo Hotel
Alindao - nearest hotel: No hotel found
Kavieng - nearest hotel: Kavieng Hotel
Avarua - nearest hotel: Paradise Inn
Lae - nearest hotel: Huon Gulf Hotel
Saint-Philippe - nearest hotel: Le Baril
Butaritari - nearest hotel: No hotel found
Tual - nearest hotel: Suita
Atuona - nearest hotel: Pearl Resort
Santa Marta - nearest hotel: Casa Mita
Maragogi - nearest hotel: Pousada Mariluz
Padang - nearest hotel: losmen Surya
Inhambane - nearest hotel: Hotel de Inhambane
Bambous Virieux - nearest hotel: No hotel found
Basco - nearest hotel: Tawsen Hotel
Olinda - nearest hotel: Fidji 

Kandrian - nearest hotel: No hotel found
Port Victoria - nearest hotel: No hotel found
Denpasar - nearest hotel: Hotel Candra Adiguna
Lakatoro - nearest hotel: No hotel found
Tezu - nearest hotel: No hotel found


,City,Country,Max_Temp,Lat,Lng,Current_Description,Hotel Name
1,Vaini,TO,75.36,-21.2000,-175.2000,moderate rain,Keleti International Resort
4,Arraial Do Cabo,BR,72.30,-22.9661,-42.0278,overcast clouds,No hotel found
11,Busselton,AU,83.32,-33.6500,115.3333,clear sky,No hotel found
17,Maridi,SS,81.61,4.9158,29.4742,few clouds,No hotel found
21,Hwange,ZW,83.57,-18.3693,26.5019,broken clouds,No hotel found
...,...,...,...,...,...,...,...
680,Kandrian,PG,81.23,-6.2167,149.5500,overcast clouds,No hotel found
690,Port Victoria,KE,74.75,0.0939,33.9756,broken clouds,No hotel found
691,Denpasar,ID,87.46,-8.6500,115.2167,moderate rain,Hotel Candra Adiguna
693,Lakatoro,VU,80.83,-16.0999,167.4164,overcast clouds,No hotel found


In [33]:
print(name_address)

{'type': 'FeatureCollection', 'features': []}


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [27]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.loc[(hotel_df["Hotel Name"] != "No hotel found")]

# Display sample data
clean_hotel_df.head(10)

,City,Country,Max_Temp,Lat,Lng,Current_Description,Hotel Name
1,Vaini,TO,75.36,-21.2000,-175.2000,moderate rain,Keleti International Resort
34,Rikitea,PF,77.56,-23.1203,-134.9692,broken clouds,Chez Bianca & Benoit
42,Cockburn Town,TC,78.37,21.4612,-71.1419,clear sky,The Salt Raker Inn
44,Kismayo,SO,83.80,-0.3582,42.5454,clear sky,Kismayo Hotel
48,Kavieng,PG,80.02,-2.5744,150.7967,moderate rain,Kavieng Hotel
51,Avarua,CK,78.85,-21.2078,-159.7750,light rain,Paradise Inn
64,Lae,PG,81.48,-6.7221,146.9847,overcast clouds,Huon Gulf Hotel
66,Saint-Philippe,RE,82.11,-21.3585,55.7679,light rain,Le Baril
73,Tual,ID,83.80,-5.6667,132.7500,overcast clouds,Suita
75,Atuona,PF,78.06,-9.8000,-139.0333,scattered clouds,Pearl Resort


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [28]:
# Create the output File (CSV)
outputfile = "WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(outputfile, index_label="City_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [32]:
# Configure the map plot
hotel_map = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Max Temp",
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name", "Country","Current_Description"]
)



# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country,Current_Description)